In [1]:
#
# LSTM모델을 이용해서 주가예측
#
import os
import settings
import pandas as pd        # 라이브러리
import numpy as np         # 라이브러리
from keras.models import Sequential      # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import Dense           # 딥러닝을 구동하는 데 필요한 케라스 함수
from keras.layers import LSTM            # 딥러닝을 구동하는 데 필요한 케라스 함수
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
import pymysql         # 파이썬에서 mysql연동시켜주는 라이브러리


# 데이터셋 생성 함수
look_back = 1
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i + look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)
 
#
# 저장되어있는 주식데이터 불러오기
#
sydtpath = os.path.join(settings.BASE_DIR, 'chart_data/%s' % (settings.get_today_str()))
stock_code = "jeju"
fullpath = sydtpath + os.path.sep + stock_code + '.csv'
pandf = pd.read_csv(fullpath, index_col="Date")

# 데이터 전처리
nparr = pandf['Close'].values[1:]     # 맨처음 'Close'데이터부터 차례대로 nparr에 저장
print(nparr)
nparr.astype('float32')    # float형으로 변환
print(nparr)
nparr = nparr.reshape(-1,1)
print(nparr)
 
# 정규화 (0~1사이의 값으로 바꿔준다)
scaler = MinMaxScaler(feature_range=(0, 1))
nptf = scaler.fit_transform(nparr)
 
# 학습용, 테스트용 데이터로 나누기 (90%를 학습용 데이터, 10%를 테스트용 데이터)
train_size = int(len(nptf) * 0.9)
test_size = len(nptf) - train_size
train, test = nptf[0:train_size], nptf[train_size:len(nptf)]
print(len(train), len(test))
 
# 학습을 위한 데이터셋 생성하기 (학습용, 테스트용으로 구분)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)
 
# RNN모델은 3차원 데이터
# trainX, testX값을 [samples, time steps, features] 형태로 reshape
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
 
# LSTM모델
model = Sequential()                                                # 딥러닝 구조, 층을 설정
model.add(LSTM(20, input_shape=(1, 1)))     # (timestep, feature)   # 층이 추가됨 (add)   # 입력층, 첫번째 은닉층                   
model.add(Dense(1))                                                 # 출력층 (하나)
# 모델을 컴파일 (컴퓨터가 알아들을 수 있도록)   # 오차함수, 최적화 방법
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])   # metrics : 모델 수행 결과를 나타내게 설정 (과적합 문제 방지)
model.fit(trainX, trainY, epochs=1000, batch_size=50, verbose=2)      # 모델을 실제로 수행     # batch_size : 전체 데이터를 10개씩 사용
                                                                                               # verbose(로깅)  2 : epoch당 나오게

# 예측값 평가하기 (얼마나 정확한지)
testPredict = model.predict(testX)
testPredict = scaler.inverse_transform(testPredict)                 # testPredict : 예측 값
testY = scaler.inverse_transform(testY)                             # testY : 실제 값
testScore = math.sqrt(mean_squared_error(testY, testPredict))       # mean_squared_error : 평균 제곱근 오차
print('Train Score: %.2f RMSE' % testScore)                         # 예측 값과 실제 값 차이 출력
 
# 예측 데이터 출력
lastX = nptf[-1]
lastX = np.reshape(lastX, (1, 1, 1))
lastY = model.predict(lastX)
predict = scaler.inverse_transform(lastY)                    # 정규화 시킨 값을 역변환
print('Predict the Close value of final day: %d' % predict)  # 데이터 입력 마지막 다음날 종가 예측
    
# 차트출력, 저장
plt.plot(testPredict)
plt.plot(testY)

plt.title('jeju predict graph')

plt.savefig("./chart_picture/jeju.png",dpi=300)
#plt.show()

# 사진 데이터 binary형식으로 바꿔주는 함수
def convertToBinaryData(filename):
    #Convert digital data to binary format
    with open(filename, 'rb') as file:
        binaryData = file.read()
    return binaryData


# 
# DB테이블 값 삽입 (INSERT)
#
# MySQL Connection 연결
connection = pymysql.connect(host='222.122.86.187', port=3306, user='geniuses777', password='stock7840',
                       db='geniuses777', charset='utf8')
try:
    with connection.cursor() as cursor:
        sql = 'INSERT INTO stock_hye (company_name, stock_price, image) VALUES (%s, %s, %s) ON DUPLICATE KEY UPDATE stock_price = VALUES(stock_price), image = VALUES(image)'
        image = convertToBinaryData("C:\source\SPF\chart_picture\jeju.png")
        cursor.execute(sql, ('제주항공', int(predict), image))          # 넣으려는 값
    connection.commit()
    
finally:
    connection.close()

Using TensorFlow backend.


[45000 42750 44700 45700 44050 42100 42000 41050 40300 39850 37850 38800
 38500 39300 40750 40950 40150 40150 39200 38900 37250 37850 38050 38000
 39850 40550 39250 39250 40850 41900 42800 41500 40500 39650 40000 39000
 40100 40900 40250 39250 37500 38200 38300 37450 36900 37150 35950 35100
 32200 29200 30950 31300 31750 32200 32800 31750 30700 30850 31750 31200
 30500 31600 32200 31250 31900 32600 33500 34250 34100 34300 33000 32750
 32500 32650 33300 33100 34950 35600 35800 35300 37150 38000 39150 38000
 37150 37300 36900 37700 37000 36850 38000 38000 37500 37000 36300 36000
 35550 34900 35500 35050 34500 35050 35000 36500 36600 36900 35900 35400
 35250 34750 33850 34550 34850 34800 34300 34600 34000 32500 32200 31650
 30700 32200 31950 32000 34000 34550 34850 34450 32850 33200 32500 32000
 32350 32100 32900 32300 32600 32100 31650 31350 31500 31200 32000 32200
 31550 31400 30850 31400 33150 33450 33350 33650 33400 32550 31850 31400
 31500 31600 31550 31500 31350 30750 31050 30800 30

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Epoch 1/1000
 - 2s - loss: 0.2462 - acc: 0.0026
Epoch 2/1000
 - 0s - loss: 0.1964 - acc: 0.0026
Epoch 3/1000
 - 0s - loss: 0.1544 - acc: 0.0026
Epoch 4/1000
 - 0s - loss: 0.1194 - acc: 0.0026
Epoch 5/1000
 - 0s - loss: 0.0910 - acc: 0.0026
Epoch 6/1000
 - 0s - loss: 0.0685 - acc: 0.0026
Epoch 7/1000
 - 0s - loss: 0.0519 - acc: 0.0026
Epoch 8/1000
 - 0s - loss: 0.0403 - acc: 0.0026
Epoch 9/1000
 - 0s - loss: 0.0331 - acc: 0.0026
Epoch 10/1000
 - 0s - loss: 0.0290 - acc: 0.0052
Epoch 11/1000
 - 0s - loss: 0.0266 - acc: 0.0052
Epoch 12/1000
 - 0s - loss: 0.0252 - acc: 0.0052
Epoch 13/1000
 - 0s - loss: 0.0241 - acc: 0.0052
Epoch 14/1000
 - 0s - loss: 0.0231 - acc: 0.0052
Epoch 15/1000
 - 0s - loss: 0.0222 - acc: 0.0052
Epoch 16/1000
 - 0s - loss: 0.0212 - acc: 0.0052
Epoch 17/1000
 - 0s - loss: 0.0203 - acc: 0.0052
Epoch 18/1000
 - 0s - loss: 0.0193 - acc: 0.0052
Epoch 19/1000
 - 0s - loss: 0.0184 - acc: 0.0052
Epoch 20/1000
 - 0s - loss: 0.0174 - acc: 0.0052
Epoch 21/1000
 - 0s - loss: 0

 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 168/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 169/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 170/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 171/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 172/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 173/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 174/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 175/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 176/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 177/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 178/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 179/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 180/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 181/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 182/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 183/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 184/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 185/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 186/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 187/1000


 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 332/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 333/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 334/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 335/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 336/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 337/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 338/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 339/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 340/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 341/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 342/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 343/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 344/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 345/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 346/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 347/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 348/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 349/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 350/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 351/1000


 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 496/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 497/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 498/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 499/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 500/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 501/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 502/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 503/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 504/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 505/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 506/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 507/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 508/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 509/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 510/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 511/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 512/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 513/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 514/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 515/1000


 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 660/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 661/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 662/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 663/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 664/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 665/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 666/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 667/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 668/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 669/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 670/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 671/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 672/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 673/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 674/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 675/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 676/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 677/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 678/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 679/1000


 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 824/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 825/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 826/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 827/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 828/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 829/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 830/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 831/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 832/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 833/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 834/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 835/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 836/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 837/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 838/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 839/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 840/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 841/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 842/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 843/1000


 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 988/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 989/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 990/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 991/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 992/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 993/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 994/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 995/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 996/1000
 - 0s - loss: 0.0011 - acc: 0.0052
Epoch 997/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 998/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 999/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Epoch 1000/1000
 - 0s - loss: 0.0010 - acc: 0.0052
Train Score: 789.96 RMSE
Predict the Close value of final day: 35250
